<a href="https://colab.research.google.com/github/kou1342/kou1342.github.io/blob/main/recall%E7%B7%8F%E5%BD%93%E3%81%9F%E3%82%8A%E6%88%A6%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ragas datasets rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.59
    Uninstalling langchain-core-0.3.59:
      Successfully uninstalled langchain-core-0.3.59


In [ ]:
import asyncio
import math
import pandas as pd
from ragas import SingleTurnSample
from ragas.metrics import NonLLMContextPrecisionWithReference

def load_group_from_csv(path: str) -> list[str]:
    """
    CSV ファイルを読み込み、1 列目の各行を文字列のリストとして返す。
    ヘッダー行がある場合は、header=0 に変更してください。
    """
    df = pd.read_csv(path, header=None, encoding='utf-8')
    return df.iloc[:, 0].dropna().astype(str).tolist()

async def compute_all_vs_all_precision(group_a: list[str], group_b: list[str]) -> dict[str, dict[str, float]]:
    """
    group_a の各要素を retrieved_contexts、group_b の各要素を reference_contexts として
    Precision スコアを計算し、辞書形式で返す。
    """
    precision_metric = NonLLMContextPrecisionWithReference()
    results: dict[str, dict[str, float]] = {}

    for a in group_a:
        row: dict[str, float] = {}
        for b in group_b:
            sample = SingleTurnSample(
                retrieved_contexts=[a],
                reference_contexts=[b]
            )
            score = await precision_metric.single_turn_ascore(sample)
            row[b] = score
        results[a] = row

    return results

def main():
    # CSV から読み込み
    group_a = load_group_from_csv("group_a.csv")
    group_b = load_group_from_csv("group_b.csv")

    # Precision 計算（非同期実行）
    results = asyncio.run(compute_all_vs_all_precision(group_a, group_b))

    # Precision==1.0 のペア数をカウント→倍カウントされる気がするから割る2してます
    total_exact_matches = sum(
        1
        for row in results.values()
        for score in row.values()
        if math.isclose(score, 1.0, rel_tol=1e-9, abs_tol=0.0)
    )/2


    # 分母は要素数が多い方
    denominator = max(len(group_a), len(group_b))
    precision_ALL = total_exact_matches / denominator if denominator > 0 else 0.0

    # 結果表示
    for a, row in results.items():
        for b, score in row.items():
            print(f"───\nA: {a}\nB: {b}\nPrecision: {score:.4f}\n")

    print(f"Precision==1.0 のペア数: {total_exact_matches}")
    print(f"要素数が多い方の数 ({denominator}) で割った precision_ALL: {precision_ALL:.4f}")

if __name__ == "__main__":
    main()

───
A: 左肩の痛みは軽減した
B: リハビリ実施中
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 痛みは解消傾向
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 患者はエプロンの着脱や棚のものも取りやすくなったと感じている
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 左肩屈曲150度
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 左肩真横120度
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 左肩外旋40度
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 左肩の指椎間距離がLの4番
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 左肩SSP5で疼痛なし
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 右肩屈曲150度
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 右肩真横120度
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 右肩外旋60度
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 右肩の指椎間距離がLの5番
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: 医師はリハビリの継続を指示した
Precision: 0.0000

───
A: 左肩の痛みは軽減した
B: １ヶ月後再診予定
Precision: 0.0000

───
A: 左肩の可動域が改善した
B: リハビリ実施中
Precision: 0.0000

───
A: 左肩の可動域が改善した
B: 痛みは解消傾向
Precision: 0.0000

───
A: 左肩の可動域が改善した
B: 患者はエプロンの着脱や棚のものも取りやすくなったと感じている
Precision: 0.0000

───
A: 左肩の可動域が改善した
B: 左肩屈曲150度
Precision: 0.0000

───
A: 左肩の可動域が改善した
B: 左肩真横120度
Precision: 0.0000

───
A:

In [ ]:
import asyncio
import math
import pandas as pd
from ragas import SingleTurnSample
from ragas.metrics import NonLLMContextRecall

def load_group_from_csv(path: str) -> list[str]:
    """
    CSV ファイルを読み込み、1 列目の各行を文字列のリストとして返す。
    ヘッダー行がある場合は header=0 に変更してください。
    """
    df = pd.read_csv(path, header=None, encoding='utf-8')
    return df.iloc[:, 0].dropna().astype(str).tolist()

async def compute_all_vs_all_recall(group_a: list[str], group_b: list[str]) -> dict[str, dict[str, float]]:
    """
    group_a の各要素を retrieved_contexts、
    group_b の各要素を reference_contexts として
    NonLLMContextRecallWithReference を用いて
    リコールスコアを計算し、辞書形式で返す。
    """
    recall_metric = NonLLMContextRecall()  #
    results: dict[str, dict[str, float]] = {}
    for a in group_a:
        row: dict[str, float] = {}
        for b in group_b:
            sample = SingleTurnSample(
                retrieved_contexts=[a],
                reference_contexts=[b]
            )
            score = await recall_metric.single_turn_ascore(sample)
            row[b] = score
        results[a] = row
    return results

def main():
    # CSV から読み込み
    group_a = load_group_from_csv("group_a.csv")
    group_b = load_group_from_csv("group_b.csv")

    # リコール計算（非同期実行）
    results = asyncio.run(compute_all_vs_all_recall(group_a, group_b))

    # recall が「ほぼ 1.0」とみなせるペア数をカウント
    total_perfect_recalls = sum(
        1
        for row in results.values()
        for score in row.values()
        if math.isclose(score, 1.0, rel_tol=1e-9)  #
    )/2

    # 分母は要素数が多い方
    denominator = max(len(group_a), len(group_b))
    recall_ALL = total_perfect_recalls / denominator if denominator > 0 else 0.0

    # 結果表示
    for a, row in results.items():
        for b, score in row.items():
            print(f"───\nA: {a}\nB: {b}\nRecall: {score:.6f}\n")

    print(f"Recall≈1.0 のペア数: {total_perfect_recalls}")
    print(f"要素数が多い方の数 ({denominator}) で割った recall_ALL: {recall_ALL:.4f}")

if __name__ == "__main__":
    main()

───
A: 左肩の痛みは軽減した
B: リハビリ実施中
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 痛みは解消傾向
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 患者はエプロンの着脱や棚のものも取りやすくなったと感じている
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 左肩屈曲150度
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 左肩真横120度
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 左肩外旋40度
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 左肩の指椎間距離がLの4番
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 左肩SSP5で疼痛なし
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 右肩屈曲150度
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 右肩真横120度
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 右肩外旋60度
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 右肩の指椎間距離がLの5番
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: 医師はリハビリの継続を指示した
Recall: 0.000000

───
A: 左肩の痛みは軽減した
B: １ヶ月後再診予定
Recall: 0.000000

───
A: 左肩の可動域が改善した
B: リハビリ実施中
Recall: 0.000000

───
A: 左肩の可動域が改善した
B: 痛みは解消傾向
Recall: 0.000000

───
A: 左肩の可動域が改善した
B: 患者はエプロンの着脱や棚のものも取りやすくなったと感じている
Recall: 0.000000

───
A: 左肩の可動域が改善した
B: 左肩屈曲150度
Recall: 0.000000

───
A: 左肩の可動域が改善した
B: 左肩真横120度
Recall: 0.000000

───
A: 左肩の可動域が改善した
B: 左肩外